# PyTorch के लिए ट्रांसफार्मर स्रोत कोड स्पष्टीकरण
[2.2-इलस्ट्रेटेड ट्रांसफार्मर] (./अध्याय 2-ट्रांसफॉर्मर संबंधित सिद्धांत/2.2-इलस्ट्रेटेड ट्रांसफार्मर.एमडी) को पढ़ने के बाद, मुझे आशा है कि आप इस खंड में ट्रांसफार्मर के प्रत्येक मॉड्यूल के डिजाइन और गणना की स्पष्ट समझ प्राप्त कर सकते हैं पाइटोरच पर आधारित हैं आइए इस जटिल मॉडल को और सीखने में सभी की मदद करने के लिए एक ट्रांसफार्मर लागू करें। 2.2.1 से भिन्न, जब यह आलेख ट्रांसफार्मर को लागू करता है, तो इसे इनपुट-मॉडल-आउटपुट के क्रम में लागू किया जाता है। संदर्भ के लिए।
**अध्याय**

- [शब्द एम्बेडिंग](#एम्बेड)
- [स्थिति एन्कोडिंग](#pos)
- [मल्टीहेड अटेंशन](#मल्टीहेड)
- [ट्रांसफार्मर बनाएं](#बिल्ड)

![](./pictures/0-1-transformer-arc.png)

चित्र: ट्रांसफार्मर संरचना आरेख

## **<div id='embed'>शब्द एम्बेडिंग</div>**

जैसा कि ऊपर चित्र में दिखाया गया है, ट्रांसफार्मर आरेख का बाईं ओर एनकोडर है, और दाईं ओर डिकोडर भाग है। एनकोडर स्रोत भाषा अनुक्रम को इनपुट करता है, और डिकोडर उस भाषा पाठ को इनपुट करता है जिसका अनुवाद करने की आवश्यकता होती है (प्रशिक्षण के दौरान)। एक पाठ अक्सर कई अनुक्रमों से बना होता है। एक सामान्य ऑपरेशन अनुक्रम पर कुछ प्रीप्रोसेसिंग करना है (जैसे शब्द विभाजन, आदि) और इसे एक सूची में बदलना है। अनुक्रम सूची के तत्व आमतौर पर शब्दावली में सबसे छोटे शब्द होते हैं जिसे खंडित नहीं किया जा सकता है। संपूर्ण पाठ एक बड़ी सूची है जिसके तत्व अनुक्रमों से बनी सूचियाँ हैं। उदाहरण के लिए, एक अनुक्रम को खंडित करने के बाद, यह ["am", "##ro", "##zi", "meets", "his", "father"] बन जाता है, और फिर उनके संबंधित अनुक्रमणिका के अनुसार शब्दावली रूपांतरण करें और मान लें कि परिणाम [23, 94, 13, 41, 27, 96] है। यदि पूरे पाठ में कुल 100 वाक्य हैं, तो उसके तत्वों के रूप में 100 सूचियाँ होंगी। क्योंकि प्रत्येक अनुक्रम की लंबाई अलग-अलग है, इसलिए अधिकतम लंबाई निर्धारित करने की आवश्यकता है संपूर्ण पाठ को एक सरणी में, आकार 100 x 128 है, जो बैच_आकार और seq_length से मेल खाता है।

इनपुट के बाद, शब्द एम्बेडिंग प्रसंस्करण किया जाता है। शब्द एम्बेडिंग प्रत्येक शब्द को पूर्व-प्रशिक्षित वेक्टर के साथ मैप करना है।

शब्द एम्बेडिंग टॉर्च पर आधारित हैं`torch.nn.Embedding` लागू किया गया है। इंस्टेंटियेटिंग करते समय जिन मापदंडों को सेट करने की आवश्यकता होती है वे हैं शब्दावली का आकार और मैप किए गए वेक्टर का आयाम, जैसे `embed = nn.Embedding(10,8)`। आम आदमी के शब्दों में, एक वेक्टर का आयाम यह है कि वेक्टर में कितनी संख्याएँ हैं। ध्यान दें कि पहला पैरामीटर शब्द सूची का आकार है। यदि आपके पास वर्तमान में 8 शब्द हैं, तो आमतौर पर 10 भरें (यदि आप बाद में इन 8 शब्दों से भिन्न शब्द दर्ज करते हैं तो एक अतिरिक्त स्थान आरक्षित है)। इसे मैप किया जाएगा। अनक करने के लिए, अनुक्रम के पैडिंग भाग को पैड पर मैप किया जाता है।

यदि हम 8 आयामों (num_features या embed_dim) पर मैप करने की योजना बनाते हैं, तो संपूर्ण पाठ का आकार 100 x 128 x 8 हो जाता है। आइए समझाने के लिए एक छोटा सा उदाहरण दें: मान लें कि हमारी शब्दावली सूची में कुल 10 शब्द हैं (अनक और पैड को गिनते हुए), और पाठ में 2 वाक्य हैं, प्रत्येक वाक्य में 4 शब्द हैं हम प्रत्येक शब्द को 8- तक मैप करना चाहते हैं। आयामी वेक्टर. तो 2, 4, और 8 बैच_साइज़, seq_length, एम्बेड_डिम के अनुरूप हैं (यदि बैच पहले आयाम में है)।

इसके अलावा, आम तौर पर गहन शिक्षण कार्य केवल num_features को बदलते हैं, इसलिए आयाम आम तौर पर उस आयाम पर आधारित होता है जहां अंतिम सुविधा स्थित होती है।

प्रोग्रामिंग प्रारंभ करें:

सभी आवश्यक पैकेजों का आयात:

In [1]:
import torch
import torch.nn as nn
from torch.nn.parameter import Parameter
from torch.nn.init import xavier_uniform_
from torch.nn.init import constant_
from torch.nn.init import xavier_normal_
import torch.nn.functional as F
from typing import Optional, Tuple, Any
from typing import List, Optional, Tuple
import math
import warnings

In [2]:
X = torch.zeros((2,4),dtype=torch.long)
embed = nn.Embedding(10,8)
print(embed(X).shape)

torch.Size([2, 4, 8])


## **<div id='pos'>स्थिति एन्कोडिंग</div>**

शब्द एम्बेडिंग के बाद स्थिति कोडिंग होती है, जिसका उपयोग विभिन्न शब्दों और एक ही शब्द की विभिन्न विशेषताओं के बीच संबंध को अलग करने के लिए किया जाता है। कोड में ध्यान दें: X_ केवल एक प्रारंभिक मैट्रिक्स है, कोई इनपुट नहीं; स्थिति एन्कोडिंग पूरी होने के बाद एक ड्रॉपआउट जोड़ा जाएगा। इसके अलावा, स्थिति एन्कोडिंग को सबसे अंत में जोड़ा जाता है, इसलिए इनपुट और आउटपुट आकार अपरिवर्तित रहते हैं।

In [3]:
Tensor = torch.Tensor
def positional_encoding(X, num_features, dropout_p=0.1, max_len=512) -> Tensor:
    r'''
        给输入加入位置编码
    参数：
        - num_features: 输入进来的维度
        - dropout_p: dropout的概率，当其为非零时执行dropout
        - max_len: 句子的最大长度，默认512
    
    形状：
        - 输入： [batch_size, seq_length, num_features]
        - 输出： [batch_size, seq_length, num_features]

    例子：
        >>> X = torch.randn((2,4,10))
        >>> X = positional_encoding(X, 10)
        >>> print(X.shape)
        >>> torch.Size([2, 4, 10])
    '''

    dropout = nn.Dropout(dropout_p)
    P = torch.zeros((1,max_len,num_features))
    X_ = torch.arange(max_len,dtype=torch.float32).reshape(-1,1) / torch.pow(
        10000,
        torch.arange(0,num_features,2,dtype=torch.float32) /num_features)
    P[:,:,0::2] = torch.sin(X_)
    P[:,:,1::2] = torch.cos(X_)
    X = X + P[:,:X.shape[1],:].to(X.device)
    return dropout(X)

In [4]:
#स्थितीय एन्कोडिंग उदाहरण
X = torch.randn((2,4,10))
X = positional_encoding(X, 10)
print(X.shape)

torch.Size([2, 4, 10])


## **<div id='multihead'>मल्टी-हेड ध्यान</div>**

### अलग करें और बैल के ध्यान तंत्र को देखें
**मल्टीहेड अटेंशन मैकेनिज्म का वर्ग जिसे पूर्ण संस्करण में चलाया जा सकता है वह पीछे है। आइए पहले पूर्ण संस्करण पर एक नजर डालें: मल्टीहेड अटेंशन मैकेनिज्म-मल्टीहेड अटेंशन अनुभाग और फिर निम्नलिखित स्पष्टीकरण पर वापस आएं। **

मल्टी-हेड अटेंशन क्लास के मुख्य घटक हैं: पैरामीटर आरंभीकरण, मल्टी_हेड_अटेंशन_फॉरवर्ड

#### आरंभीकरण पैरामीटर
```अजगर
यदि self._qkv_same_embed_dim ग़लत है:
    #प्रारंभिकरण से पहले और बाद में आकार अपरिवर्तित रहता है
    # (seq_length x एम्बेड_डिम) x (एम्बेड_डिम x एम्बेड_डिम) ==> (seq_length x एम्बेड_डिम)
    self.q_proj_weight = पैरामीटर(मशाल.खाली((एम्बेड_डिम, एंबेड_डिम)))
    self.k_proj_weight = पैरामीटर(मशाल.खाली((embed_dim, self.kdim)))
    self.v_proj_weight = पैरामीटर(मशाल.खाली((embed_dim, self.vdim)))
    self.register_parameter('in_proj_weight', कोई नहीं)
अन्य:
    self.in_proj_weight = पैरामीटर(मशाल.खाली((3 * एम्बेड_डिम, एम्बेड_डिम)))
    self.register_parameter('q_proj_weight', कोई नहीं)
    self.register_parameter('k_proj_weight', कोई नहीं)
    self.register_parameter('v_proj_weight', कोई नहीं)

यदि पूर्वाग्रह:
    self.in_proj_bias = पैरामीटर(मशाल.खाली(3 * एंबेड_डिम))
अन्य:
    self.register_parameter('in_proj_bias', कोई नहीं)
# बाद के चरण में, सभी प्रमुखों का ध्यान एक साथ लगाया जाएगा और फिर वेट मैट्रिक्स से गुणा करके आउटपुट दिया जाएगा
#आउट_प्रोज बाद की तैयारी के लिए है
self.out_proj= एनएन.रैखिक(एम्बेड_डिम, एंबेड_डिम, पूर्वाग्रह=पूर्वाग्रह)
self._reset_parameters()
```

मशाल.खाली दिए गए आकार के अनुसार संबंधित टेंसर बनाता है। विशेषता यह है कि भरे हुए मान को मशाल.रैंडन (मानक सामान्य वितरण) के अनुरूप प्रारंभ नहीं किया गया है, यह एक आरंभीकरण विधि है। PyTorch में, यदि चर प्रकार टेंसर है, तो मान को संशोधित नहीं किया जा सकता है, और पैरामीटर() फ़ंक्शन को एक प्रकार रूपांतरण फ़ंक्शन के रूप में माना जा सकता है, जो अपरिवर्तनीय टेंसर को प्रशिक्षित और संशोधित मॉडल पैरामीटर में परिवर्तित करता है, यानी मॉडल के समान। पैरामीटर एक साथ बंधे हैं, रजिस्टर_पैरामीटर का मतलब है कि इस पैरामीटर को मॉडल.पैरामीटर में रखा जाए या नहीं, और कोई नहीं का मतलब है कि ऐसा कोई पैरामीटर नहीं है।

यह निर्धारित करने के लिए यहां एक निर्णय है कि क्या q, k, v का अंतिम आयाम सुसंगत है, यदि वे सुसंगत हैं, तो एक बड़े वजन मैट्रिक्स को गुणा किया जाएगा और फिर विभाजित किया जाएगा, वास्तव में, आरंभीकरण मूल आकार नहीं बदलेगा (जैसे कि ![](http://latex.codecogs.com/svg.latex?q=qW_q+b_q), टिप्पणियाँ देखें)।

आप पा सकते हैं कि अंत में एक _reset_parameters() फ़ंक्शन है,इसका उपयोग पैरामीटर मान आरंभ करने के लिए किया जाता है। xavier_uniform का अर्थ है [निरंतर समान वितरण](https://zh.wikipedia.org/wiki/%E9%80%A3%E7%BA%8C%E5%9E%8B%E5%9D%87%E5% 8B% BB%E5%88%86%E5%B8%83) आरंभीकरण मान के रूप में यादृच्छिक रूप से नमूना मान, और xavier_normal_ का नमूना वितरण एक सामान्य वितरण है। सटीक रूप से क्योंकि तंत्रिका नेटवर्क को प्रशिक्षित करते समय आरंभीकरण मूल्य बहुत महत्वपूर्ण है, इन दो कार्यों की आवश्यकता होती है।

स्थिरांक_ का अर्थ है इनपुट वेक्टर को दिए गए मान से भरना।

इसके अलावा, PyTorch के स्रोत कोड में, ऐसा लगता है कि प्रक्षेपण एक रैखिक परिवर्तन का प्रतिनिधित्व करता है, और in_proj_bias का अर्थ प्रारंभिक रैखिक परिवर्तन का पूर्वाग्रह है।

```अजगर
def _reset_parameters(स्वयं):
    यदि self._qkv_same_embed_dim:
        xavier_uniform_(self.in_proj_weight)
    अन्य:
        xavier_uniform_(self.q_proj_weight)
        xavier_uniform_(self.k_proj_weight)
        xavier_uniform_(self.v_proj_weight)
    यदि self.in_proj_bias कोई नहीं है:
        स्थिर_(self.in_proj_bias, 0.)
        स्थिर_(self.out_proj.bias, 0.)

```

#### मल्टी_हेड_अटेंशन_फॉरवर्ड
यह फ़ंक्शन निम्नलिखित कोड में दिखाया गया है, जिसे मुख्य रूप से 3 भागों में विभाजित किया गया है:
- क्यू, के, वी प्राप्त करने के लिए क्वेरी, कुंजी, मान को _in_projection_packed के माध्यम से रूपांतरित किया जाता है
- अवरोधन तंत्र
- बिंदु एकाग्रता

In [5]:
import torch
Tensor = torch.Tensor
def multi_head_attention_forward(
    query: Tensor,
    key: Tensor,
    value: Tensor,
    num_heads: int,
    in_proj_weight: Tensor,
    in_proj_bias: Optional[Tensor],
    dropout_p: float,
    out_proj_weight: Tensor,
    out_proj_bias: Optional[Tensor],
    training: bool = True,
    key_padding_mask: Optional[Tensor] = None,
    need_weights: bool = True,
    attn_mask: Optional[Tensor] = None,
    use_seperate_proj_weight = None,
    q_proj_weight: Optional[Tensor] = None,
    k_proj_weight: Optional[Tensor] = None,
    v_proj_weight: Optional[Tensor] = None,
) -> Tuple[Tensor, Optional[Tensor]]:
    r'''
    形状：
        输入：
        - query：`(L, N, E)`
        - key: `(S, N, E)`
        - value: `(S, N, E)`
        - key_padding_mask: `(N, S)`
        - attn_mask: `(L, S)` or `(N * num_heads, L, S)`
        输出：
        - attn_output:`(L, N, E)`
        - attn_output_weights:`(N, L, S)`
    '''
    tgt_len, bsz, embed_dim = query.shape
    src_len, _, _ = key.shape
    head_dim = embed_dim // num_heads
    q, k, v = _in_projection_packed(query, key, value, in_proj_weight, in_proj_bias)

    if attn_mask is not None:
        if attn_mask.dtype == torch.uint8:
            warnings.warn("Byte tensor for attn_mask in nn.MultiheadAttention is deprecated. Use bool tensor instead.")
            attn_mask = attn_mask.to(torch.bool)
        else:
            assert attn_mask.is_floating_point() or attn_mask.dtype == torch.bool, \
                f"Only float, byte, and bool types are supported for attn_mask, not {attn_mask.dtype}"

        if attn_mask.dim() == 2:
            correct_2d_size = (tgt_len, src_len)
            if attn_mask.shape != correct_2d_size:
                raise RuntimeError(f"The shape of the 2D attn_mask is {attn_mask.shape}, but should be {correct_2d_size}.")
            attn_mask = attn_mask.unsqueeze(0)
        elif attn_mask.dim() == 3:
            correct_3d_size = (bsz * num_heads, tgt_len, src_len)
            if attn_mask.shape != correct_3d_size:
                raise RuntimeError(f"The shape of the 3D attn_mask is {attn_mask.shape}, but should be {correct_3d_size}.")
        else:
            raise RuntimeError(f"attn_mask's dimension {attn_mask.dim()} is not supported")

    if key_padding_mask is not None and key_padding_mask.dtype == torch.uint8:
        warnings.warn("Byte tensor for key_padding_mask in nn.MultiheadAttention is deprecated. Use bool tensor instead.")
        key_padding_mask = key_padding_mask.to(torch.bool)
    
# क्यू, के, वी को पुनः आकार दें, डॉट उत्पाद के ध्यान को फिट करने के लिए बैच को पहले आयाम में रखें
# यह एक ही समय में एक बहु-सिर तंत्र है, जो एक परत बनाने के लिए अलग-अलग सिरों को एक साथ रखता है
    q = q.contiguous().view(tgt_len, bsz * num_heads, head_dim).transpose(0, 1)
    k = k.contiguous().view(-1, bsz * num_heads, head_dim).transpose(0, 1)
    v = v.contiguous().view(-1, bsz * num_heads, head_dim).transpose(0, 1)
    if key_padding_mask is not None:
        assert key_padding_mask.shape == (bsz, src_len), \
            f"expecting key_padding_mask shape of {(bsz, src_len)}, but got {key_padding_mask.shape}"
        key_padding_mask = key_padding_mask.view(bsz, 1, 1, src_len).   \
            expand(-1, num_heads, -1, -1).reshape(bsz * num_heads, 1, src_len)
        if attn_mask is None:
            attn_mask = key_padding_mask
        elif attn_mask.dtype == torch.bool:
            attn_mask = attn_mask.logical_or(key_padding_mask)
        else:
            attn_mask = attn_mask.masked_fill(key_padding_mask, float("-inf"))
# यदि attn_mask मान एक बूलियन मान है, तो मास्क को फ़्लोट में बदलें
    if attn_mask is not None and attn_mask.dtype == torch.bool:
        new_attn_mask = torch.zeros_like(attn_mask, dtype=torch.float)
        new_attn_mask.masked_fill_(attn_mask, float("-inf"))
        attn_mask = new_attn_mask

# प्रशिक्षण सही होने पर ही ड्रॉपआउट लागू करें
    if not training:
        dropout_p = 0.0
    attn_output, attn_output_weights = _scaled_dot_product_attention(q, k, v, attn_mask, dropout_p)
    attn_output = attn_output.transpose(0, 1).contiguous().view(tgt_len, bsz, embed_dim)
    attn_output = nn.functional.linear(attn_output, out_proj_weight, out_proj_bias)
    if need_weights:
# सिर पर औसत ध्यान भार
        attn_output_weights = attn_output_weights.view(bsz, num_heads, tgt_len, src_len)
        return attn_output, attn_output_weights.sum(dim=1) / num_heads
    else:
        return attn_output, None

##### क्वेरी, कुंजी, मान को q, k, v प्राप्त करने के लिए _in_projection_packed के माध्यम से रूपांतरित किया जाता है
```
q, k, v = _in_projection_packed(क्वेरी, कुंजी, मान, in_proj_weight, in_proj_bias)
```

`nn.functional.linear` फ़ंक्शन के लिए, यह वास्तव में एक रैखिक परिवर्तन है। `nn.Linear` से भिन्न, पूर्व एक भार मैट्रिक्स और पूर्वाग्रह प्रदान कर सकता है, और निष्पादित कर सकता है![](http://latex.codecogs. com/ svg.latex?y=xW^T+b), और बाद वाला आउटपुट का आयाम है जिसे स्वतंत्र रूप से निर्धारित किया जा सकता है।

In [6]:
def _in_projection_packed(
    q: Tensor,
    k: Tensor,
    v: Tensor,
    w: Tensor,
    b: Optional[Tensor] = None,
) -> List[Tensor]:
    r"""
    用一个大的权重参数矩阵进行线性变换

    参数:
        q, k, v: 对自注意来说，三者都是src；对于seq2seq模型，k和v是一致的tensor。
                 但它们的最后一维(num_features或者叫做embed_dim)都必须保持一致。
        w: 用以线性变换的大矩阵，按照q,k,v的顺序压在一个tensor里面。
        b: 用以线性变换的偏置，按照q,k,v的顺序压在一个tensor里面。

    形状:
        输入:
        - q: shape:`(..., E)`，E是词嵌入的维度（下面出现的E均为此意）。
        - k: shape:`(..., E)`
        - v: shape:`(..., E)`
        - w: shape:`(E * 3, E)`
        - b: shape:`E * 3` 

        输出:
        - 输出列表 :`[q', k', v']`，q,k,v经过线性变换前后的形状都一致。
    """
    E = q.size(-1)
# यदि यह आत्म-ध्यान है, तो q = k = v = src, इसलिए उनके संदर्भ चर सभी src हैं
# अर्थात्, k का परिणाम v है और q का k दोनों सत्य हैं
# यदि यह seq2seq, k = v है, तो k का परिणाम v सत्य है
    if k is v:
        if q is k:
            return F.linear(q, w, b).chunk(3, dim=-1)
        else:
# seq2seq मॉडल
            w_q, w_kv = w.split([E, E * 2])
            if b is None:
                b_q = b_kv = None
            else:
                b_q, b_kv = b.split([E, E * 2])
            return (F.linear(q, w_q, b_q),) + F.linear(k, w_kv, b_kv).chunk(2, dim=-1)
    else:
        w_q, w_k, w_v = w.chunk(3)
        if b is None:
            b_q = b_k = b_v = None
        else:
            b_q, b_k, b_v = b.chunk(3)
        return F.linear(q, w_q, b_q), F.linear(k, w_k, b_k), F.linear(v, w_v, b_v)

# q, k, v = _in_projection_packed(क्वेरी, कुंजी, मान, in_proj_weight, in_proj_bias)

***

##### रोड़ा तंत्र

Attn_mask के लिए, यदि यह 2D है, तो आकार `(L, S)` जैसा है, L और S क्रमशः लक्ष्य भाषा और स्रोत भाषा अनुक्रम लंबाई का प्रतिनिधित्व करते हैं, यदि यह 3D है, तो आकार `(N * num_heads, L) जैसा है , S)` , N बैच_आकार का प्रतिनिधित्व करता है, num_heads ध्यान शीर्षों की संख्या का प्रतिनिधित्व करता है। यदि attn_mask का dtype ByteTensor है, तो गैर-0 स्थितियों पर ध्यान नहीं दिया जाएगा; यदि यह BoolTensor है, तो True से संबंधित स्थितियों पर ध्यान नहीं दिया जाएगा, यदि यह एक संख्यात्मक मान है, तो इसे सीधे attn_weights में जोड़ा जाएगा .

क्योंकि जब डिकोडर डिकोड कर रहा होता है, तो वह केवल स्थिति और उससे पहले वाली स्थिति को देख सकता है। यदि आप पीछे की ओर देखते हैं, तो यह उल्लंघन होगा, इसलिए इसे attn_mask द्वारा अवरुद्ध करने की आवश्यकता है।

निम्नलिखित फ़ंक्शन सीधे PyTorch की प्रतिलिपि बनाता है, जिसका अर्थ है कि यह सुनिश्चित करना कि विभिन्न आयामों के मुखौटा आकार सही हैं और विभिन्न प्रकार के रूपांतरण हैं


```अजगर
यदि attn_mask कोई नहीं है:
    यदि attn_mask.dtype == torch.uint8:
        warnings.warn('nn.MultiheadAttention में attn_mask के लिए बाइट टेंसर को हटा दिया गया है।इसके बजाय बूल टेंसर का उपयोग करें।")
        attn_mask = attn_mask.to(torch.bool)
    अन्य:
        attn_mask.is_floating_point() या attn_mask.dtype == torch.bool, \ पर जोर दें
            f"केवल फ्लोट, बाइट और बूल प्रकार attn_mask के लिए समर्थित हैं, {attn_mask.dtype} के लिए नहीं"
    # विभिन्न आयामों में आकृतियों का निर्धारण
    यदि attn_mask.dim() == 2:
        सही_2d_आकार = (tgt_len, src_len)
        यदि attn_mask.shape != सही_2d_आकार:
            raise RuntimeError(f"2D attn_mask का आकार {attn_mask.shape} है, लेकिन { होना चाहिएसही_2d_आकार}.")
            attn_mask = attn_mask.unsqueeze(0)
    एलिफ़ attn_mask.dim() == 3:
        सही_3डी_आकार = (बीएसजेड * num_heads, tgt_len, src_len)
        यदि attn_mask.shape != सही_3d_आकार:
            raise RuntimeError(f"3D attn_mask का आकार {attn_mask.shape} है, लेकिन {correct_3d_size} होना चाहिए।")
    अन्य:
        raise RuntimeError(f"attn_mask का आयाम {attn_mask.dim()} समर्थित नहीं है")

```
`attn_mask` से भिन्न, `key_padding_mask` का उपयोग कुंजी में मान को ब्लॉक करने के लिए किया जाता है, विशेष रूप से, यह `<PAD>` होना चाहिए, जिसे अनदेखा किया जाता है।attn_mask के अनुरूप।

```अजगर
# key_padding_mask मान को बूलियन मान में बदलें
यदि key_padding_mask कोई नहीं है और key_padding_mask.dtype == torch.uint8:
    चेतावनियाँ.चेतावनी('nn.MultiheadAttention में key_padding_mask के लिए बाइट टेंसर अप्रचलित है। इसके बजाय बूल टेंसर का उपयोग करें।')
    key_padding_mask = key_padding_mask.to(torch.bool)
```

सबसे पहले दो छोटे फ़ंक्शन, `लॉजिकल_ऑर` का परिचय दें, जो दो टेंसरों को इनपुट करता है और दो टेंसरों में मानों पर `लॉजिकल OR` ऑपरेशन करता है। यह केवल तभी `गलत` होता है जब दोनों मान 0 होते हैं। और अन्य बार दोनों `True` हैं, और दूसरा `masked_fill` है। इनपुट एक मास्क है और भरने के लिए उपयोग किया जाने वाला मान है। मास्क में 1, 0 होते हैं, 0 का स्थिति मान अपरिवर्तित रहता है, और 1 की स्थिति एक नए मान से भर जाती है।
```अजगर
ए = टॉर्च.टेंसर([0,1,10,0],dtype=torch.int8)
बी = टॉर्च.टेंसर([4,0,1,0],डीटाइप=टॉर्च.इंट8)
प्रिंट(मशाल.लॉजिकल_या(ए,बी))
# टेंसर([सच्चा, सच्चा, सच्चा, गलत])
```

```अजगर
आर = टॉर्च.टेंसर([[0,0,0,0],[0,0,0,0]])
मास्क = टॉर्च.टेंसर([[1,1,1,1],[0,0,0,0]])
प्रिंट(आर.मास्कड_फिल(मास्क,1))
# टेंसर([[1, 1, 1, 1],
# [0, 0, 0, 0]])
```
वास्तव में, attn_mask और key_padding_mask में कभी-कभी समान ऑब्जेक्ट होते हैं, इसलिए कभी-कभी उन्हें एक साथ देखा जा सकता है। सॉफ्टमैक्स के लिए `-inf` का उपयोग करने के बाद, मान 0 है, जिसे अनदेखा कर दिया जाता है।
```अजगर
यदि key_padding_mask कोई नहीं है:
    ज़ोर key_padding_mask.shape == (bsz, src_len), \
        च"उदा{(bsz, src_len)} के key_padding_mask आकार की अपेक्षा की, लेकिन {key_padding_mask.shape}" प्राप्त किया
    key_padding_mask = key_padding_mask.view(bsz, 1, 1, src_len).\
        विस्तार(-1, num_heads, -1, -1).reshape(bsz * num_heads, 1, src_len)
    # यदि attn_mask खाली है, तो सीधे key_padding_mask का उपयोग करें
    यदि attn_mask कोई नहीं है:
        attn_mask = key_padding_mask
    एलिफ़ attn_mask.dtype == मशाल.बूल:
        attn_mask = attn_mask.ological_or(key_padding_mask)
    अन्य:
        attn_mask = attn_mask.masked_fill(key_padding_mask, फ़्लोट("-inf"))

# यदि attn_mask मान एक बूलियन मान है, तो मास्क को फ़्लोट में बदलें
यदि attn_mask कोई नहीं है और attn_mask.dtype == torch.bool:
    new_attn_mask = मशाल.शून्य_लाइक(attn_mask, dtype=torch.float)
    new_attn_mask.masked_fill_(attn_mask, फ्लोट("-inf"))
    attn_mask = new_attn_mask

```

***
##### डॉट उत्पाद ध्यान

In [7]:
from typing import Optional, Tuple, Any
def _scaled_dot_product_attention(
    q: Tensor,
    k: Tensor,
    v: Tensor,
    attn_mask: Optional[Tensor] = None,
    dropout_p: float = 0.0,
) -> Tuple[Tensor, Tensor]:
    r'''
    在query, key, value上计算点积注意力，若有注意力遮盖则使用，并且应用一个概率为dropout_p的dropout

    参数：
        - q: shape:`(B, Nt, E)` B代表batch size， Nt是目标语言序列长度，E是嵌入后的特征维度
        - key: shape:`(B, Ns, E)` Ns是源语言序列长度
        - value: shape:`(B, Ns, E)`与key形状一样
        - attn_mask: 要么是3D的tensor，形状为:`(B, Nt, Ns)`或者2D的tensor，形状如:`(Nt, Ns)`

        - Output: attention values: shape:`(B, Nt, E)`，与q的形状一致;attention weights: shape:`(B, Nt, Ns)`
    
    例子：
        >>> q = torch.randn((2,3,6))
        >>> k = torch.randn((2,4,6))
        >>> v = torch.randn((2,4,6))
        >>> out = scaled_dot_product_attention(q, k, v)
        >>> out[0].shape, out[1].shape
        >>> torch.Size([2, 3, 6]) torch.Size([2, 3, 4])
    '''
    B, Nt, E = q.shape
    q = q / math.sqrt(E)
# (बी, एनटी, ई) एक्स (बी, ई, एनएस) -> (बी, एनटी, एनएस)
    attn = torch.bmm(q, k.transpose(-2,-1))
    if attn_mask is not None:
        attn += attn_mask 
#attn का अर्थ है कि लक्ष्य अनुक्रम का प्रत्येक शब्द स्रोत भाषा अनुक्रम पर ध्यान देता है
    attn = F.softmax(attn, dim=-1)
    if dropout_p:
        attn = F.dropout(attn, p=dropout_p)
# (बी, एनटी, एनएस) एक्स (बी, एनएस, ई) -> (बी, एनटी, ई)
    output = torch.bmm(attn, v)
    return output, attn 


### संपूर्ण मल्टी-हेड अटेंशन मैकेनिज्म-मल्टीहेडअटेंशन

In [8]:
class MultiheadAttention(nn.Module):
    r'''
    参数：
        embed_dim: 词嵌入的维度
        num_heads: 平行头的数量
        batch_first: 若`True`，则为(batch, seq, feture)，若为`False`，则为(seq, batch, feature)
    
    例子：
        >>> multihead_attn = MultiheadAttention(embed_dim, num_heads)
        >>> attn_output, attn_output_weights = multihead_attn(query, key, value)
    '''
    def __init__(self, embed_dim, num_heads, dropout=0., bias=True,
                 kdim=None, vdim=None, batch_first=False) -> None:
# फ़ैक्टरी_क्वार्ग्स = {'डिवाइस': डिवाइस, 'डीटाइप': डीटाइप}
        super(MultiheadAttention, self).__init__()
        self.embed_dim = embed_dim
        self.kdim = kdim if kdim is not None else embed_dim
        self.vdim = vdim if vdim is not None else embed_dim
        self._qkv_same_embed_dim = self.kdim == embed_dim and self.vdim == embed_dim

        self.num_heads = num_heads
        self.dropout = dropout
        self.batch_first = batch_first
        self.head_dim = embed_dim // num_heads
        assert self.head_dim * num_heads == self.embed_dim, "embed_dim must be divisible by num_heads"

        if self._qkv_same_embed_dim is False:
            self.q_proj_weight = Parameter(torch.empty((embed_dim, embed_dim)))
            self.k_proj_weight = Parameter(torch.empty((embed_dim, self.kdim)))
            self.v_proj_weight = Parameter(torch.empty((embed_dim, self.vdim)))
            self.register_parameter('in_proj_weight', None)
        else:
            self.in_proj_weight = Parameter(torch.empty((3 * embed_dim, embed_dim)))
            self.register_parameter('q_proj_weight', None)
            self.register_parameter('k_proj_weight', None)
            self.register_parameter('v_proj_weight', None)

        if bias:
            self.in_proj_bias = Parameter(torch.empty(3 * embed_dim))
        else:
            self.register_parameter('in_proj_bias', None)
        self.out_proj = nn.Linear(embed_dim, embed_dim, bias=bias)

        self._reset_parameters()

    def _reset_parameters(self):
        if self._qkv_same_embed_dim:
            xavier_uniform_(self.in_proj_weight)
        else:
            xavier_uniform_(self.q_proj_weight)
            xavier_uniform_(self.k_proj_weight)
            xavier_uniform_(self.v_proj_weight)

        if self.in_proj_bias is not None:
            constant_(self.in_proj_bias, 0.)
            constant_(self.out_proj.bias, 0.)



    def forward(self, query: Tensor, key: Tensor, value: Tensor, key_padding_mask: Optional[Tensor] = None,
                need_weights: bool = True, attn_mask: Optional[Tensor] = None) -> Tuple[Tensor, Optional[Tensor]]:
        if self.batch_first:
            query, key, value = [x.transpose(1, 0) for x in (query, key, value)]

        if not self._qkv_same_embed_dim:
            attn_output, attn_output_weights = multi_head_attention_forward(
                query, key, value, self.num_heads,
                self.in_proj_weight, self.in_proj_bias,
                self.dropout, self.out_proj.weight, self.out_proj.bias,
                training=self.training,
                key_padding_mask=key_padding_mask, need_weights=need_weights,
                attn_mask=attn_mask, use_separate_proj_weight=True,
                q_proj_weight=self.q_proj_weight, k_proj_weight=self.k_proj_weight,
                v_proj_weight=self.v_proj_weight)
        else:
            attn_output, attn_output_weights = multi_head_attention_forward(
                query, key, value, self.num_heads,
                self.in_proj_weight, self.in_proj_bias,
                self.dropout, self.out_proj.weight, self.out_proj.bias,
                training=self.training,
                key_padding_mask=key_padding_mask, need_weights=need_weights,
                attn_mask=attn_mask)
        if self.batch_first:
            return attn_output.transpose(1, 0), attn_output_weights
        else:
            return attn_output, attn_output_weights

इसके बाद, आप इसका अभ्यास कर सकते हैं और स्थिति कोडिंग जोड़ सकते हैं। आप पा सकते हैं कि स्थिति कोडिंग और मल्टी-हेड ध्यान जोड़ने से पहले और बाद का आकार नहीं बदलेगा।

In [9]:
# क्योंकि बैच_फर्स्ट गलत है, src का आकार: `(seq, बैच, एम्बेड_डिम)`
src = torch.randn((2,4,100))
src = positional_encoding(src,100,0.1)
print(src.shape)
multihead_attn = MultiheadAttention(100, 4, 0.1)
attn_output, attn_output_weights = multihead_attn(src,src,src)
print(attn_output.shape, attn_output_weights.shape)

#मशाल.आकार([2,4,100])
# टॉर्च.आकार([2,4,100]) टॉर्च.आकार([4,2,2])

torch.Size([2, 4, 100])
torch.Size([2, 4, 100]) torch.Size([4, 2, 2])


***
## **<div id='build'>बिल्ड ट्रांसफार्मर</div>**
-एनकोडर परत

![](./pictures/2-2-1-encoder.png)

In [10]:
class TransformerEncoderLayer(nn.Module):
    r'''
    参数：
        d_model: 词嵌入的维度（必备）
        nhead: 多头注意力中平行头的数目（必备）
        dim_feedforward: 全连接层的神经元的数目，又称经过此层输入的维度（Default = 2048）
        dropout: dropout的概率（Default = 0.1）
        activation: 两个线性层中间的激活函数，默认relu或gelu
        lay_norm_eps: layer normalization中的微小量，防止分母为0（Default = 1e-5）
        batch_first: 若`True`，则为(batch, seq, feture)，若为`False`，则为(seq, batch, feature)（Default：False）

    例子：
        >>> encoder_layer = TransformerEncoderLayer(d_model=512, nhead=8)
        >>> src = torch.randn((32, 10, 512))
        >>> out = encoder_layer(src)
    '''

    def __init__(self, d_model, nhead, dim_feedforward=2048, dropout=0.1, activation=F.relu,
                 layer_norm_eps=1e-5, batch_first=False) -> None:
        super(TransformerEncoderLayer, self).__init__()
        self.self_attn = MultiheadAttention(d_model, nhead, dropout=dropout, batch_first=batch_first)
        self.linear1 = nn.Linear(d_model, dim_feedforward)
        self.dropout = nn.Dropout(dropout)
        self.linear2 = nn.Linear(dim_feedforward, d_model)

        self.norm1 = nn.LayerNorm(d_model, eps=layer_norm_eps)
        self.norm2 = nn.LayerNorm(d_model, eps=layer_norm_eps)
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)
        self.activation = activation        


    def forward(self, src: Tensor, src_mask: Optional[Tensor] = None, src_key_padding_mask: Optional[Tensor] = None) -> Tensor:
        src = positional_encoding(src, src.shape[-1])
        src2 = self.self_attn(src, src, src, attn_mask=src_mask, 
        key_padding_mask=src_key_padding_mask)[0]
        src = src + self.dropout1(src2)
        src = self.norm1(src)
        src2 = self.linear2(self.dropout(self.activation(self.linear1(src))))
        src = src + self.dropout(src2)
        src = self.norm2(src)
        return src


In [11]:
# आइए इसे एक छोटे से उदाहरण से देखें
encoder_layer = TransformerEncoderLayer(d_model=512, nhead=8)
src = torch.randn((32, 10, 512))
out = encoder_layer(src)
print(out.shape)
#मशाल.आकार([32,10,512])

torch.Size([32, 10, 512])


### ट्रांसफार्मर परत एनकोडर बनाती है

In [12]:
class TransformerEncoder(nn.Module):
    r'''
    参数：
        encoder_layer（必备）
        num_layers： encoder_layer的层数（必备）
        norm: 归一化的选择（可选）
    
    例子：
        >>> encoder_layer = TransformerEncoderLayer(d_model=512, nhead=8)
        >>> transformer_encoder = TransformerEncoder(encoder_layer, num_layers=6)
        >>> src = torch.randn((10, 32, 512))
        >>> out = transformer_encoder(src)
    '''

    def __init__(self, encoder_layer, num_layers, norm=None):
        super(TransformerEncoder, self).__init__()
        self.layer = encoder_layer
        self.num_layers = num_layers
        self.norm = norm
    
    def forward(self, src: Tensor, mask: Optional[Tensor] = None, src_key_padding_mask: Optional[Tensor] = None) -> Tensor:
        output = positional_encoding(src, src.shape[-1])
        for _ in range(self.num_layers):
            output = self.layer(output, src_mask=mask, src_key_padding_mask=src_key_padding_mask)
        
        if self.norm is not None:
            output = self.norm(output)
        
        return output

In [13]:
# उदाहरण
encoder_layer = TransformerEncoderLayer(d_model=512, nhead=8)
transformer_encoder = TransformerEncoder(encoder_layer, num_layers=6)
src = torch.randn((10, 32, 512))
out = transformer_encoder(src)
print(out.shape)
#मशाल.आकार([10,32,512])

torch.Size([10, 32, 512])


***
## डिकोडर परत:

In [14]:
class TransformerDecoderLayer(nn.Module):
    r'''
    参数：
        d_model: 词嵌入的维度（必备）
        nhead: 多头注意力中平行头的数目（必备）
        dim_feedforward: 全连接层的神经元的数目，又称经过此层输入的维度（Default = 2048）
        dropout: dropout的概率（Default = 0.1）
        activation: 两个线性层中间的激活函数，默认relu或gelu
        lay_norm_eps: layer normalization中的微小量，防止分母为0（Default = 1e-5）
        batch_first: 若`True`，则为(batch, seq, feture)，若为`False`，则为(seq, batch, feature)（Default：False）
    
    例子：
        >>> decoder_layer = TransformerDecoderLayer(d_model=512, nhead=8)
        >>> memory = torch.randn((10, 32, 512))
        >>> tgt = torch.randn((20, 32, 512))
        >>> out = decoder_layer(tgt, memory)
    '''
    def __init__(self, d_model, nhead, dim_feedforward=2048, dropout=0.1, activation=F.relu,
                 layer_norm_eps=1e-5, batch_first=False) -> None:
        super(TransformerDecoderLayer, self).__init__()
        self.self_attn = MultiheadAttention(d_model, nhead, dropout=dropout, batch_first=batch_first)
        self.multihead_attn = MultiheadAttention(d_model, nhead, dropout=dropout, batch_first=batch_first)

        self.linear1 = nn.Linear(d_model, dim_feedforward)
        self.dropout = nn.Dropout(dropout)
        self.linear2 = nn.Linear(dim_feedforward, d_model)

        self.norm1 = nn.LayerNorm(d_model, eps=layer_norm_eps)
        self.norm2 = nn.LayerNorm(d_model, eps=layer_norm_eps)
        self.norm3 = nn.LayerNorm(d_model, eps=layer_norm_eps)
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)
        self.dropout3 = nn.Dropout(dropout)

        self.activation = activation

    def forward(self, tgt: Tensor, memory: Tensor, tgt_mask: Optional[Tensor] = None, 
                memory_mask: Optional[Tensor] = None,tgt_key_padding_mask: Optional[Tensor] = None, memory_key_padding_mask: Optional[Tensor] = None) -> Tensor:
        r'''
        参数：
            tgt: 目标语言序列（必备）
            memory: 从最后一个encoder_layer跑出的句子（必备）
            tgt_mask: 目标语言序列的mask（可选）
            memory_mask（可选）
            tgt_key_padding_mask（可选）
            memory_key_padding_mask（可选）
        '''
        tgt2 = self.self_attn(tgt, tgt, tgt, attn_mask=tgt_mask,
                              key_padding_mask=tgt_key_padding_mask)[0]
        tgt = tgt + self.dropout1(tgt2)
        tgt = self.norm1(tgt)
        tgt2 = self.multihead_attn(tgt, memory, memory, attn_mask=memory_mask,
                                   key_padding_mask=memory_key_padding_mask)[0]
        tgt = tgt + self.dropout2(tgt2)
        tgt = self.norm2(tgt)
        tgt2 = self.linear2(self.dropout(self.activation(self.linear1(tgt))))
        tgt = tgt + self.dropout3(tgt2)
        tgt = self.norm3(tgt)
        return tgt

In [15]:
# प्यारा सा उदाहरण
decoder_layer = nn.TransformerDecoderLayer(d_model=512, nhead=8)
memory = torch.randn((10, 32, 512))
tgt = torch.randn((20, 32, 512))
out = decoder_layer(tgt, memory)
print(out.shape)
#मशाल.आकार([20,32,512])

torch.Size([20, 32, 512])


In [16]:
##डिकोडर

In [17]:
class TransformerDecoder(nn.Module):
    r'''
    参数：
        decoder_layer（必备）
        num_layers: decoder_layer的层数（必备）
        norm: 归一化选择
    
    例子：
        >>> decoder_layer =TransformerDecoderLayer(d_model=512, nhead=8)
        >>> transformer_decoder = TransformerDecoder(decoder_layer, num_layers=6)
        >>> memory = torch.rand(10, 32, 512)
        >>> tgt = torch.rand(20, 32, 512)
        >>> out = transformer_decoder(tgt, memory)
    '''
    def __init__(self, decoder_layer, num_layers, norm=None):
        super(TransformerDecoder, self).__init__()
        self.layer = decoder_layer
        self.num_layers = num_layers
        self.norm = norm
    
    def forward(self, tgt: Tensor, memory: Tensor, tgt_mask: Optional[Tensor] = None,
                memory_mask: Optional[Tensor] = None, tgt_key_padding_mask: Optional[Tensor] = None,
                memory_key_padding_mask: Optional[Tensor] = None) -> Tensor:
        output = tgt
        for _ in range(self.num_layers):
            output = self.layer(output, memory, tgt_mask=tgt_mask,
                         memory_mask=memory_mask,
                         tgt_key_padding_mask=tgt_key_padding_mask,
                         memory_key_padding_mask=memory_key_padding_mask)
        if self.norm is not None:
            output = self.norm(output)

        return output

In [18]:
# प्यारा सा उदाहरण
decoder_layer =TransformerDecoderLayer(d_model=512, nhead=8)
transformer_decoder = TransformerDecoder(decoder_layer, num_layers=6)
memory = torch.rand(10, 32, 512)
tgt = torch.rand(20, 32, 512)
out = transformer_decoder(tgt, memory)
print(out.shape)
#मशाल.आकार([20,32,512])

torch.Size([20, 32, 512])


संक्षेप में कहें तो, वास्तव में, स्थिति एन्कोडिंग और मल्टी-हेड ध्यान के बाद, एनकोडर परत और डिकोडर परत का आकार नहीं बदलेगा, और एनकोडर और डिकोडर क्रमशः src और tgt के आकार के अनुरूप हैं।

## ट्रांसफार्मर

In [19]:
class Transformer(nn.Module):
    r'''
    参数：
        d_model: 词嵌入的维度（必备）（Default=512）
        nhead: 多头注意力中平行头的数目（必备）（Default=8）
        num_encoder_layers:编码层层数（Default=8）
        num_decoder_layers:解码层层数（Default=8）
        dim_feedforward: 全连接层的神经元的数目，又称经过此层输入的维度（Default = 2048）
        dropout: dropout的概率（Default = 0.1）
        activation: 两个线性层中间的激活函数，默认relu或gelu
        custom_encoder: 自定义encoder（Default=None）
        custom_decoder: 自定义decoder（Default=None）
        lay_norm_eps: layer normalization中的微小量，防止分母为0（Default = 1e-5）
        batch_first: 若`True`，则为(batch, seq, feture)，若为`False`，则为(seq, batch, feature)（Default：False）
    
    例子：
        >>> transformer_model = Transformer(nhead=16, num_encoder_layers=12)
        >>> src = torch.rand((10, 32, 512))
        >>> tgt = torch.rand((20, 32, 512))
        >>> out = transformer_model(src, tgt)
    '''
    def __init__(self, d_model: int = 512, nhead: int = 8, num_encoder_layers: int = 6,
                 num_decoder_layers: int = 6, dim_feedforward: int = 2048, dropout: float = 0.1,
                 activation = F.relu, custom_encoder: Optional[Any] = None, custom_decoder: Optional[Any] = None,
                 layer_norm_eps: float = 1e-5, batch_first: bool = False) -> None:
        super(Transformer, self).__init__()
        if custom_encoder is not None:
            self.encoder = custom_encoder
        else:
            encoder_layer = TransformerEncoderLayer(d_model, nhead, dim_feedforward, dropout,
                                                    activation, layer_norm_eps, batch_first)
            encoder_norm = nn.LayerNorm(d_model, eps=layer_norm_eps)
            self.encoder = TransformerEncoder(encoder_layer, num_encoder_layers)

        if custom_decoder is not None:
            self.decoder = custom_decoder
        else:
            decoder_layer = TransformerDecoderLayer(d_model, nhead, dim_feedforward, dropout,
                                                    activation, layer_norm_eps, batch_first)
            decoder_norm = nn.LayerNorm(d_model, eps=layer_norm_eps)
            self.decoder = TransformerDecoder(decoder_layer, num_decoder_layers, decoder_norm)

        self._reset_parameters()

        self.d_model = d_model
        self.nhead = nhead

        self.batch_first = batch_first

    def forward(self, src: Tensor, tgt: Tensor, src_mask: Optional[Tensor] = None, tgt_mask: Optional[Tensor] = None,
                memory_mask: Optional[Tensor] = None, src_key_padding_mask: Optional[Tensor] = None,
                tgt_key_padding_mask: Optional[Tensor] = None, memory_key_padding_mask: Optional[Tensor] = None) -> Tensor:
        r'''
        参数：
            src: 源语言序列（送入Encoder）（必备）
            tgt: 目标语言序列（送入Decoder）（必备）
            src_mask: （可选)
            tgt_mask: （可选）
            memory_mask: （可选）
            src_key_padding_mask: （可选）
            tgt_key_padding_mask: （可选）
            memory_key_padding_mask: （可选）
        
        形状：
            - src: shape:`(S, N, E)`, `(N, S, E)` if batch_first.
            - tgt: shape:`(T, N, E)`, `(N, T, E)` if batch_first.
            - src_mask: shape:`(S, S)`.
            - tgt_mask: shape:`(T, T)`.
            - memory_mask: shape:`(T, S)`.
            - src_key_padding_mask: shape:`(N, S)`.
            - tgt_key_padding_mask: shape:`(N, T)`.
            - memory_key_padding_mask: shape:`(N, S)`.

            [src/tgt/memory]_mask确保有些位置不被看到，如做decode的时候，只能看该位置及其以前的，而不能看后面的。
            若为ByteTensor，非0的位置会被忽略不做注意力；若为BoolTensor，True对应的位置会被忽略；
            若为数值，则会直接加到attn_weights

            [src/tgt/memory]_key_padding_mask 使得key里面的某些元素不参与attention计算，三种情况同上

            - output: shape:`(T, N, E)`, `(N, T, E)` if batch_first.

        注意：
            src和tgt的最后一维需要等于d_model，batch的那一维需要相等
            
        例子:
            >>> output = transformer_model(src, tgt, src_mask=src_mask, tgt_mask=tgt_mask)
        '''
        memory = self.encoder(src, mask=src_mask, src_key_padding_mask=src_key_padding_mask)
        output = self.decoder(tgt, memory, tgt_mask=tgt_mask, memory_mask=memory_mask,
                              tgt_key_padding_mask=tgt_key_padding_mask,
                              memory_key_padding_mask=memory_key_padding_mask)
        return output
        
    def generate_square_subsequent_mask(self, sz: int) -> Tensor:
        r'''产生关于序列的mask，被遮住的区域赋值`-inf`，未被遮住的区域赋值为`0`'''
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask

    def _reset_parameters(self):
        r'''用正态分布初始化参数'''
        for p in self.parameters():
            if p.dim() > 1:
                xavier_uniform_(p)

In [20]:
# छोटा सा उदाहरण
transformer_model = Transformer(nhead=16, num_encoder_layers=12)
src = torch.rand((10, 32, 512))
tgt = torch.rand((20, 32, 512))
out = transformer_model(src, tgt)
print(out.shape)
#मशाल.आकार([20,32,512])

torch.Size([20, 32, 512])


अब तक, हमने PyTorch की ट्रांसफॉर्मर लाइब्रेरी को पूरी तरह से कार्यान्वित किया है, आधिकारिक संस्करण की तुलना में, हस्तलिखित संस्करण में कम निर्णय कथन हैं।
## आभार
यह लेख ताई युनपेंग द्वारा लिखा गया था और इस परियोजना के सदस्यों द्वारा पुनर्गठित और संगठित किया गया था। अंत में, मैं आपकी पढ़ने की प्रतिक्रिया और सितारों की प्रतीक्षा कर रहा हूं, धन्यवाद।